## Get started with ADK Redis Memory Service

#### Set Google Cloud project information

In [ ]:
PROJECT_ID = !gcloud config get-value project
PROJECT_ID = PROJECT_ID[0]

LOCATION = !gcloud config get-value compute/region
LOCATION = LOCATION[0]

PROJECT_ID, LOCATION

#### Set environment variables required for ADK

In [ ]:
import os

os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "TRUE"
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION

#### Define a simple agent

The ADK Runner defines what session service is used by the agent. The session service is responsible for saving the conversation history.

In this example, we're using the `redis` (or `valkey`)  session service (`RedisSessionService`).

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai.types import Content, Part

APP_NAME = "redis_memory_service"
MODEL = "gemini-2.5-flash"

agent = Agent(
    model=MODEL,
    name="Generic_QA_Agent",
    instruction="Answer the user's questions",
)

#### Helpers

In [ ]:
from typing import (
  Any,
  Dict
)


async def call_agent_async(
  runner: Runner,
  user_id: str,
  session_id: str,
  prompt: str
) -> Dict[str, Any]:
  """Call agent asynchronously and return response with token usage."""

  response_parts = []
  token_usage = {
      "prompt_token_count": 0,
      "candidates_token_count": 0,
      "cached_content_token_count": 0,
      "total_token_count": 0,
  }

  async for event in runner.run_async(
      user_id=user_id,
      session_id=session_id,
      new_message=Content(parts=[Part(text=prompt)], role="user"),
  ):
    if event.content and event.content.parts:
      for part in event.content.parts:
        if hasattr(part, "text") and part.text:
          response_parts.append(part.text)

    # Collect token usage information
    if event.usage_metadata:
      if (
          hasattr(event.usage_metadata, "prompt_token_count")
          and event.usage_metadata.prompt_token_count
      ):
        token_usage[
            "prompt_token_count"
        ] += event.usage_metadata.prompt_token_count
      if (
          hasattr(event.usage_metadata, "candidates_token_count")
          and event.usage_metadata.candidates_token_count
      ):
        token_usage[
            "candidates_token_count"
        ] += event.usage_metadata.candidates_token_count
      if (
          hasattr(event.usage_metadata, "cached_content_token_count")
          and event.usage_metadata.cached_content_token_count
      ):
        token_usage[
            "cached_content_token_count"
        ] += event.usage_metadata.cached_content_token_count
      if (
          hasattr(event.usage_metadata, "total_token_count")
          and event.usage_metadata.total_token_count
      ):
        token_usage[
            "total_token_count"
        ] += event.usage_metadata.total_token_count

  response_text = "".join(response_parts)

  return {"response_text": response_text, "token_usage": token_usage}

### Create a Session and Store a Simple Conversation

In [ ]:
session_service = InMemorySessionService()

### Generating Memories

In [ ]:
USER_ID = 'user'

chat_session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID
)

In [ ]:
runner = Runner(agent=agent, app_name=APP_NAME, session_service=session_service)

In [ ]:
agent_response = await call_agent_async(
    runner,
    prompt="what types of questions do you answer",
    session_id=chat_session.id,
    user_id=USER_ID
)

In [ ]:
chat_session = await session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id=chat_session.id
)

In [ ]:
recommendations_session = await session_service.create_session(
    app_name=APP_NAME, user_id=USER_ID
)

await call_agent_async(
    runner,
    prompt="I have a three year old niece. What recommendations do you have for birthday presents?",
    session_id=recommendations_session.id,
    user_id=USER_ID,
)
await call_agent_async(
    runner,
    prompt="I like the idea of a bike. Are there any age appropriate options?",
    session_id=recommendations_session.id,
    user_id=USER_ID,
)

recommendations_session = await session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id=recommendations_session.id
)

### RedisMemoryService

In [ ]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
   sys.path.insert(0, project_root)

from redis_memory_service.lib.redis_memory_service import RedisMemoryService

In [ ]:
redis_memory_service = RedisMemoryService(
    uri="redis://localhost:6379",
    index_name="adk_app_memory",
    embedding_model_name= "gemini-embedding-001",
)

In [ ]:
await redis_memory_service.add_session_to_memory(chat_session)
await redis_memory_service.add_session_to_memory(recommendations_session)

In [ ]:
APP_NAME = 'redis_memory_service'
USER_ID = 'user'

response = await redis_memory_service.search_memory(
    app_name=APP_NAME,
    user_id=USER_ID,
    query="What was I thinking about getting my niece for Christmas?"
)

response

#### Inspecting Session Data

As you interact with the agent, its session state will be stored in your Redis database. You can inspect this data.

In [ ]:
import redis

redis_client = redis.Redis(host='localhost', port=6379, db=0)

In [ ]:
modules = redis_client.module_list()
module_name_list = [m[b'name'] for m in modules]
if b'search' in module_name_list:
    print("RediSearch module is loaded.")
else:
    print("RediSearch module is not loaded. Please ensure it's enabled in your Redis configuration.")

In [ ]:
indexes = redis_client.execute_command("FT._LIST")

if indexes:
    print("Redis Vector Indexes:")
    for index_name in indexes:
        print(f"- {index_name.decode('utf-8')}")
        # Optional
        info = redis_client.execute_command("FT.INFO", index_name)
        print(f"  Info: {info}")
else:
    print("No Redis vector indexes found.")